# Topic Modelling : trouver des thèmes dans un corpus

Le topic modelling est un outil de machine learning non supervisé.

Il permet de trouver les thèmes majeurs dans un corpus de textes, de façon automatique (ou presque).

Pour appliquer le topic modelling, **gensim** est la librairie de référence
![](https://github.com/stat4decision/ml-dl-d3s-avr2021/blob/main/images/gensim.png?raw=1)

IL faut d'abord l'installer, à l'aide de la commande suivante :
```
conda install -c anaconda gensim 
```

Il existe plusieurs modèle de Topic Modelling, les plus connus sont :
- LSA pour Latent Semantic Analysis, qui fonctionne un peu comme une analyse en composantes principales
- LDA pour Latent Dirichlet Allocation, qui fonctionne un peu comme un K-means

Nous allons ici nous focaliser sur la mise en oeuvre de la LDA, mais la méthodologie est la même pour la LSA : 
- Preprocessing des données textuelles : calcul du BOW et/ou TF-IDF
- Entrainement du modèle (e.g. LDA ou LSA)
- Interprétation des résultats et définition des thèmes majeurs
- Eventuelles itérations pour affiner les résultats

### Preprocessing des données

Nous allons réutiliser le dataset des revues amazon, et calculer le TF-IDF :

In [1]:
from google.colab import files
uploaded = files.upload()

In [2]:
import pandas as pd

df = pd.read_csv('amazon_reviews.csv', sep='|')
df.head()

,review,class
0,great cd: my lovely pat has one of the great v...,1
1,one of the best game music soundtracks - for a...,1
2,batteries died within a year ...: i bought thi...,0
3,"works fine, but maha energy is better: check o...",1
4,great for the non-audiophile: reviewed quite a...,1


In [ ]:
!pip install pyldavis 

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
stops = stopwords.words('english')
nltk.download('punkt')


def clean_data(quote):
    quote = quote.lower()
    tokens = word_tokenize(quote)
    token_punc = [t for t in tokens if t.isalpha()]
    token_stop = [t for t in token_punc if t not in stops]
    return token_stop

# On effectue le traitement en tokens pour la suite
df['tokens'] = df['review'].apply(clean_data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# On calcule de TF-IDF sur les 1000 premières reviews
vectorizer = TfidfVectorizer(stop_words=stops, analyzer=lambda x: x)

tfidf = vectorizer.fit_transform(df['tokens'])

### Topic modelling

Nous allons ensuite passer à l'étape de Topic Modelling à l'aide de gensim.

La première étape est toujours la même : instancier le modèle de LDA qu'il faut importer au préalable de la façon suivante :
```python
from gensim.models import LdaModel
```

La classe `LdaModel` a la signature suivante :
```python
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            id2word,
                                            num_topics, 
                                            random_state,
                                            chunksize,
                                            passes)
```

Avec :
- `Corpus` le TF-IDF ou BOW (calculé avec scikit-learn)
- `id2word` un dictionnaire avec les correspondances entre indices dans le corpus et mots (calculé par gensim)
- `num_topics` est le nombre désiré de thèmes
- `random_state` pour la reproductibilité (mettre toujours la même valeur)
- `chunksize` la taille du mini-batch (laisser par défaut)
- `passes` le nombre de fois que le modèle verra tout le corpus pour s'entrainer (1 par défaut, en général 5 ou 10 est très bien)


La seule information qui nous manque est donc le paramètre à donner en `id2word`.

On peut utiliser pour ça l'objet `Dictionary`, qui s'importe de la façon suivante 
```python
from gensim.corpora import Dictionary
```

Et qui s'applique directement sur les tokens (d'où la nécessité de le calculer au-dessus) :

In [5]:
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize

id2word = Dictionary(df['tokens'])

Enfin un dernier détail avant de pouvoir utiliser le modèle, il faut convertir le TF-IDF de `scikit-learn` en un format propre à `gensim` à l'aide de la fonction `Sparse2Corpus` :

In [6]:
from gensim.matutils import Sparse2Corpus

tfidf_gensim = Sparse2Corpus(tfidf, documents_columns=False)

Finalement, il est possible d'instancier et d'entrainer notre modèle (le tout se faisant en même temps) :

In [7]:
from gensim.models import LdaModel

# On instancie et entraine un modèle qui trouvera 3 thèmes
lda = LdaModel(corpus=tfidf_gensim, id2word=id2word, num_topics=3, random_state=0, passes=5)

Il est maintenant possible d'afficher les thèmes majeurs, avec la méthode `.print_topics()` :

In [8]:
from pprint import pprint
pprint(lda.print_topics())

[(0,
  '0.007*"obligatory" + 0.006*"thats" + 0.004*"binaries" + 0.004*"bcsn" + '
  '0.003*"sacrilege" + 0.002*"pony" + 0.002*"chopping" + 0.002*"smiling" + '
  '0.001*"conditions" + 0.001*"broke"'),
 (1,
  '0.001*"covers" + 0.001*"fx" + 0.001*"jagged" + 0.001*"disability" + '
  '0.001*"emanuell" + 0.001*"exceeption" + 0.001*"aikido" + 0.000*"sly" + '
  '0.000*"engineers" + 0.000*"positioning"'),
 (2,
  '0.006*"cronilogical" + 0.004*"occupy" + 0.004*"dolbly" + 0.003*"hatred" + '
  '0.003*"awestruck" + 0.003*"barker" + 0.003*"mcad" + 0.003*"hercules" + '
  '0.002*"texans" + 0.002*"fillers"')]


L'idée est ensuite d'interpréter ces résultats...

### Visualisation des résultats

Il existe une librairie dédiée de visualisation des résultats de LDA, nommée `pyLDAvis`.

Cette dernière doit au préalable être installée avec la commande suivante :
```
conda install -c conda-forge pyldavis 
```

L'utilisation est ensuite relativement simple, il suffit ensuite d'effectuer quelques imports et lignes de code :

In [16]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
bow = [id2word.doc2bow(line) for line in df['tokens']]  # convert corpus to BoW format
vis = pyLDAvis.gensim_models.prepare(topic_model=lda, dictionary=id2word, corpus=bow)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.126983  0.004406       1        1  45.074637
0     -0.048714 -0.030600       2        1  29.803067
1     -0.078270  0.026194       3        1  25.122297, topic_info=               Term         Freq        Total Category  logprob  loglift
4208     obligatory   777.000000   777.000000  Default  30.0000  30.0000
687           thats   708.000000   708.000000  Default  29.0000  29.0000
3026   cronilogical  1159.000000  1159.000000  Default  28.0000  28.0000
25905          bcsn   467.000000   467.000000  Default  27.0000  27.0000
18293      binaries   547.000000   547.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
4560       suffered    32.574300    50.725607   Topic3  -8.0112   0.9385
22161         autos    19.011164    24.576958   Topic3  -8.5497   1.1246
20885     psychotic    30.169712    53.157822   Topic3  -8.0879   0.8150
16574   superfluous    22.479007    32.738866   Topic3  -8.3822   1.0054
16373     fantasies    21.455156    34.260928   Topic3  -8.4288   0.9134

[179 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
29022      1  0.017463  abnormal
29022      2  0.925551  abnormal
29022      3  0.052390  abnormal
18196      1  0.900789  absentee
18196      2  0.091151  absentee
...      ...       ...       ...
1973       2  0.135379      weak
1973       3  0.009906      weak
10915      1  0.935185    weaker
10915      2  0.049220    weaker
10915      3  0.014766    weaker

[429 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

**Exercice :**

Appliquer le topic modelling au dataset `headlines.csv`, qui contient des titres de journaux.

In [ ]:
df = pd.read_csv('data/headlines.csv')
df.head()